In [64]:
import pandas as pd
import geopandas as gpd
import censusdata

In [158]:
geo3 = gpd.read_file("2010 Census Tracts.geojson")

In [163]:
# data: https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku

In [74]:
geo2 = gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/\
arcgis/rest/services/NYC_Census_Tracts_for_2010_US_Census_Water_Included/\
FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")

In [65]:
geo = gpd.read_file("gz_2010_36_140_00_500k/gz_2010_36_140_00_500k.shp")

In [90]:
geo2.shape

(2168, 13)

In [23]:
# Get population estimates from census

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

pop = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '36'), ('county', '*'), ('tract', '*')]),['B01003_001E','GEO_ID'])
pop = pop[['GEO_ID','B01003_001E']].rename(columns = {'B01003_001E':'total_pop'})
pop['county_tract'] = pop['GEO_ID'].str.slice(start =11, stop = 20)
pop['state_county_tract'] = pop['GEO_ID'].str.slice(start =9, stop = 20)
pop_clean = pop.drop(['GEO_ID'], axis = 1).reset_index(drop = True)


In [145]:
pop_clean = pop.reset_index(drop = True)
pop_clean['CT2010'] = pop_clean.county_tract.str.slice(start=3,stop=11)
pop_clean['County'] = pop_clean.county_tract.str.slice(start=0,stop=3)

In [150]:
pop_clean.loc[pop_clean.County == "005",'County'] = "2"
pop_clean.loc[pop_clean.County == "061",'County'] = "1"
pop_clean.loc[pop_clean.County == "047",'County'] = "3"
pop_clean.loc[pop_clean.County == "081",'County'] = "4"
pop_clean.loc[pop_clean.County == "085",'County'] = "5"

pop_clean['BoroCT2010'] = pop_clean['County'] + pop_clean['CT2010']

In [151]:
test2 = geo2.merge(pop_clean, on = "BoroCT2010")

In [160]:
test3 = geo3.merge(pop_clean, left_on = "boro_ct2010", right_on = "BoroCT2010")

In [162]:
test3.shape

(2165, 20)

In [156]:
test.shape

(4898, 15)

In [153]:
test2.shape

(2167, 20)

In [86]:
test[test.county_tract == "103170101"].head(2)

,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,geometry,total_pop,county_tract,state_county_tract
4715,1400000US36103170101,36,103,170101,1701.01,Tract,0.96,"POLYGON ((-72.37235 41.10078, -72.37218 41.100...",2082,103170101,36103170101


In [155]:
test = geo.merge(pop_clean, on="GEO_ID")

In [69]:
test[test.COUNTY.isin(['005','061','047','081','085'])].to_file("test.json", driver="GeoJSON")

In [157]:
test2.to_file("test2.json", driver="GeoJSON")

In [161]:
test3.to_file("test3.json", driver="GeoJSON")